<a href="https://colab.research.google.com/github/theventurecity/data-toolkit/blob/master/Mini_Pipeline_MAU_Growth_Accounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![TheVentureCity](https://theventure.city/wp-content/uploads/2017/06/Theventurecity-logoweb-1.png)

# Mini-Pipeline: MAU Growth Accounting
1. Extract raw event log data from a CSV
2. Transform that data into growth accounting analysis dataframes
3. Load the transformed data into Google Sheets 
4. Visualize insights in Google Data Studio

## Before you begin

- This notebook is shared with read-only access. To run this notebook yourself, first click "**Open in Playground**" in the toolbar above. That will create a separate instance that you can run and/or save a copy of to your own Google Drive. 

- To run each cell, hit **Shift-Enter**, which will run the contents of the active cell and move to the next cell. This includes the markup cells (such as this one).

- When you run the first block of Python code, you will get a message that says, "**Warning: This notebook was not authored by Google.**" Please be aware that we are **NOT** accessing your data shared with Google or reading data and credentials from other sessions. This notebook reads data from GitHub and writes to a Google Sheet that only you have access to and can control. We recommend you click the box to "**Reset all runtimes before running**" for extra information security.

## Import relevant Python libraries

In [0]:
### Pandas, Numpy, and date functions to read the data from its source
### and manipulate it in memory
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import datetime
import math


### The IPython.display library allows us to embed an iFrame within this 
### notebook
from IPython.display import IFrame


### To run this with functions from TheVentureCity's GitHub repository,
### clone the repository to the Google Colaboratory runtime environment
### and then import the code. This code allows us to run pre-existing functions 
### rather than having to define them inline within the notebook
### THIS IS ONLY APPLICABLE IF YOU WANT TO ACCESS THOSE FUNCTIONS ###
from importlib.machinery import SourceFileLoader
!git clone https://github.com/theventurecity/data-toolkit.git /tmp/theventurecity
!mv /tmp/theventurecity/python/tvc_load_colab.py tvc_load_colab.py
!mv /tmp/theventurecity/python/tvc_transform.py tvc_transform.py
!rm -r /tmp/theventurecity
tvcl = SourceFileLoader('tvc_load_colab', 'tvc_load_colab.py').load_module()
tvct = SourceFileLoader('tvc_transform', 'tvc_transform.py').load_module()

Cloning into '/tmp/theventurecity'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 64 (delta 21), reused 43 (delta 8), pack-reused 0
Unpacking objects: 100% (64/64), done.


## 1. Extract raw event log data from a CSV

This example uses a data file for a sample company from our GitHub repository called ServBiz. In this step we read the data file into memory as a Pandas dataframe we name "t."

In [0]:
# Edit this filename to your local filename.csv if using a local CSV file
filename = 'https://raw.githubusercontent.com/theventurecity/analytics/master/data/ServBiz_transactions.csv'

t = pd.read_csv(filename)
t.tail(10)

,client_id,date,value_usd,segment
420781,27902A,2019-02-28,8.7500,B2B
420782,34181A,2019-02-28,18.9700,B2C
420783,30168A,2019-02-28,17.7250,B2C
420784,30844A,2019-02-28,19.9750,B2C
420785,35815A,2019-02-28,17.9750,B2C
420786,16958A,2019-02-28,17.4500,B2C
420787,13090A,2019-02-28,13.4750,B2C
420788,19162A,2019-02-28,13.6375,B2B
420789,28409A,2019-02-28,14.7250,B2C
420790,12080A,2019-02-28,18.3250,B2C


## 2. Transform the raw data into growth accounting analysis dataframes
**Note**: For a more detailed discussion about creating the DAU and DAU Decorated dataframes, complete with inline code, visit [Section 2 of Mini-Pipeline: Cohort Analysis](https://colab.research.google.com/drive/1oYy-wJl6VZFgOsv8uw7iGChQxUjrR5rf#scrollTo=aTeIz9GPwYV1)
### 2.1 Create Daily Active Users (DAU) dataframe
The **DAU** dataframe aggregates all activity by user and day. 

In [0]:
# Run the create_dau_df function and show the first ten rows of the resulting dataframe
dau = tvct.create_dau_df(t, 
                    user_id = 'client_id', 
                    activity_date = 'date', 
                    inc_amt = 'value_usd')
dau.head(10)

,user_id,activity_date,inc_amt
0,10000A,2015-10-14,11.75
1,10001A,2015-10-14,13.75
2,10001A,2015-11-02,7.50
3,10001A,2015-11-22,18.00
4,10001A,2017-04-04,6.25
5,10001A,2017-12-08,8.75
6,10002A,2015-10-14,11.75
7,10002A,2015-10-26,12.25
8,10002A,2015-11-16,12.25
9,10002A,2015-11-23,12.25


### 2.2 - 2.3 Calculate First Date and DAU Decorated dataframes
The create_dau_decorated_df function calls the create_first_dt_df if no first_dt dataframe is specified

In [0]:
# Run the create_dau_decorated_df function and show the first ten rows of the resulting dataframe
dau_decorated = tvct.create_dau_decorated_df(dau)
dau_decorated.head(10)

Creating DAU Decorated dataframe
Creating first_dt dataframe


,user_id,activity_date,inc_amt,first_dt,first_week,first_month
0,10000A,2015-10-14,11.75,2015-10-14,2015-10-12/2015-10-18,2015-10
1,10001A,2015-10-14,13.75,2015-10-14,2015-10-12/2015-10-18,2015-10
2,10001A,2015-11-02,7.50,2015-10-14,2015-10-12/2015-10-18,2015-10
3,10001A,2015-11-22,18.00,2015-10-14,2015-10-12/2015-10-18,2015-10
4,10001A,2017-04-04,6.25,2015-10-14,2015-10-12/2015-10-18,2015-10
5,10001A,2017-12-08,8.75,2015-10-14,2015-10-12/2015-10-18,2015-10
6,10002A,2015-10-14,11.75,2015-10-14,2015-10-12/2015-10-18,2015-10
7,10002A,2015-10-26,12.25,2015-10-14,2015-10-12/2015-10-18,2015-10
8,10002A,2015-11-16,12.25,2015-10-14,2015-10-12/2015-10-18,2015-10
9,10002A,2015-11-23,12.25,2015-10-14,2015-10-12/2015-10-18,2015-10


Combining the basic DAU data with the first date, week, and month for each user, **the DAU Decorated dataframe is our basic building block for many different analyses**. It allows us to use user-level data to inspect engagement, retention, and growth accounting.

### 2.4 Calculate MAU growth accounting metrics
**Note**: The code in this section has been modified from what is in the tvc_transform.py file. This is for the sake of brevity and simplicity.

Now that we have the "DAU Decorated" data frame, we can use it to calculate growth accounting metrics since they are super-important for an early-stage startup. As you can see by reviewing the code comments below, knowing a customer's first month helps us understand, in any given month, if we have ever seen that customer before. That information allows us to distinguish between "new" and "resurrected" users.
#### First, aggregate DAU Decorated into MAU Decorated by calculating monthly totals for each user

In [0]:
### For discrete time period calculations, this helps set the variable names
### in the different dataframes 
def get_time_period_dict(time_period):
    
    time_fields_dict = {
                        'day' : {'grouping_col' : 'activity_date',
                                  'first_period_col' : 'first_dt',
                                  'frequency' : 'Daily',
                                  'unit' : 'Day',
                                  'period_abbr' : 'D',
                                  'python_period' : 'days',
                                  'days' : 1
                                  },
                        'week' : {'grouping_col' : 'Week',
                                  'first_period_col' : 'first_week',
                                  'frequency' : 'Weekly',
                                  'unit' : 'Week',
                                  'period_abbr' : 'W',
                                  'python_period' : 'weeks',
                                  'days' : 7
                                  },
                        'month' : {'grouping_col' : 'Month_Year',
                                   'first_period_col' : 'first_month',
                                   'frequency' : 'Monthly',
                                   'unit' : 'Month',
                                   'period_abbr' : 'M',
                                   'python_period' : 'months',
                                   'days' : 28
                                  }
                        }
                    
    # if time_period passed in is a valid choice, then return the dictionary
    # associated with that choice from the dictionary above
    if time_period in time_fields_dict:
        time_fields = time_fields_dict[time_period]
    else:
        time_fields = None
    
    return time_fields
  


### This is another helper function that allows us to determine the next week
### or month for any given week of month. We need these because the Pandas
### date math is not consistent. You can use timedelta to add weeks, but you 
### have to use DateOffset to add months.
def increment_period(xau_grouping_col, time_period):
  
    # Call the get_time_period_dict function above, passing in the time period
    time_fields = get_time_period_dict(time_period)
    
    # Set the one-letter period abbreviation to whatever that function returns
    period_abbr = time_fields['period_abbr']
    
    # Depending on the time period, increment the week or month by one after
    # first converting it to the date time of the start of the period
    if time_period == 'week':
        start_of_next_period = pd.to_datetime(pd.PeriodIndex(xau_grouping_col).start_time + timedelta(weeks = 1))
    elif time_period == 'month':
        start_of_next_period = pd.to_datetime(pd.PeriodIndex(xau_grouping_col, freq = period_abbr).start_time) + pd.DateOffset(months = 1)
    else:
        start_of_next_period = None
        
    # Convert the date time from the previous step back into the appropriate 
    # period (week or month) 
    if start_of_next_period is not None:
        next_period = pd.Series(start_of_next_period).dt.to_period(period_abbr)
    else:
        next_period = None
    
    # Return the next period
    return next_period



### create_xau_decorated_df is a generic function that allows us to find WAU
### Decorated or MAU Decorated from a DAU Decorated based on the time period
### that gets passed in
def create_xau_decorated_df(dau_decorated_df, time_period):
    
    # These are the parameters that are set from the get_time_period_dict 
    # function above
    time_fields = get_time_period_dict(time_period)
    grouping_col = time_fields['grouping_col']
    frequency = time_fields['frequency']
    first_period_col = time_fields['first_period_col']
    period_abbr = time_fields['period_abbr']
    
    # Print a notification message indicating that this function has been called
    print('Creating ' + frequency + ' Active Users Decorated dataframe')
    
    # We are grouping by the grouping_col (which is either "Week" or "Month_Year"),
    # the user_id, and the first_period_col (either "first_week" or "first_month")
    # For each user_id, there is one and only one first_period_col
    groupby_cols = [grouping_col, 'user_id', first_period_col]
        
    # Start by making a copy of the dataframe that gets passed in so as not to
    # affect the original
    dau_decorated = dau_decorated_df.copy()
    
    # Convert the activity_date for each transaction in dau_decorated to a 
    # period the same timeframe as the period in question (either a week or a 
    # month)
    dau_decorated[grouping_col] = pd.to_datetime(dau_decorated['activity_date']).dt.to_period(period_abbr)
    
    # Group dau_decorated into the grouping_cols defined above and aggregate the
    # sum of the inc_amt field
    xau = (dau_decorated.groupby(groupby_cols, as_index = False)['inc_amt'].sum())
    
    # Set a new column with the next time period by calling the increment_period
    # function defined above
    xau['Next_' + grouping_col] = increment_period(xau[grouping_col], time_period)
    
    # Select a subset of the resultant columns from the groupby to output
    output_cols = [grouping_col, 'user_id', 'inc_amt', first_period_col, 'Next_' + grouping_col]
    xau = xau[output_cols]
    
    # Return the resultant dataset
    return xau
  
  

# Run the create_xau_decorated_df function above (which calls the other two 
# functions in this code block) and show the first ten rows
mau_decorated = create_xau_decorated_df(dau_decorated, 'month')
mau_decorated.head(10)

Creating Monthly Active Users Decorated dataframe


,Month_Year,user_id,inc_amt,first_month,Next_Month_Year
0,2015-05,9030A,101.8125,2015-05,2015-06
1,2015-05,9033A,7.5000,2015-05,2015-06
2,2015-05,9036A,25.5000,2015-05,2015-06
3,2015-05,9045A,11.7500,2015-05,2015-06
4,2015-05,9046A,48.7500,2015-05,2015-06
5,2015-05,9051A,11.7500,2015-05,2015-06
6,2015-05,9056A,85.7500,2015-05,2015-06
7,2015-05,9057A,23.5000,2015-05,2015-06
8,2015-05,9062A,11.7500,2015-05,2015-06
9,2015-05,9064A,12.2500,2015-05,2015-06


#### Next, calculate total users by category in each month
Now that we have aggregated each user's activity in each month, we can start calculating the growth accounting for each month. In this step, we are calculating the following:

* **Monthly Active Users (MAU)** used the service this month
* **Retained** users used the service both this month and last month
* **New** users have never used the service before this month
* **Resurrected** users used the service before this month, but did not use it last month
* **Churned** users used the service last month but not this month 


In [0]:
### This takes a dataframe of transactions grouped by a particular date period
### and returns active, retained, new, resurrected, and churned users for that
### time period
### Reminder: the .t suffix stands for "this month" and .l stands for "last month"
def calc_user_ga(x, grouping_col, first_period_col):
  
    # au = Active Users = the count of unique user_id's this period
    au = x.loc[~x[grouping_col + '.t'].isnull(), 'user_id'].nunique() 
    
    # ret_users = retained_users = unique user_ids that transacted this period 
    # and last
    ret_users = (x.loc[(x['inc_amt.t'] > 0) & (x['inc_amt.l'] > 0), 
                       'user_id']
                 .nunique())
    
    # new_users = unique user_id's for whom  this period's first_period_col is 
    # the same as this period's grouping_col
    new_users = (x.loc[x[first_period_col + '.t'] == x[grouping_col + '.t'], 
                       'user_id']
                 .nunique())
    
    # res_users = resurrected users = transacted this period but not last, and
    # this is not their first period to transact
    res_users = (x.loc[(x[first_period_col + '.t'] != x[grouping_col + '.t']) & 
                       ~(x['inc_amt.l'] > 0), 
                       'user_id']
                 .nunique())
    
    # churned_users = transacted last period but not this one
    churned_users = -1 * x.loc[~(x['inc_amt.t'] > 0), 'user_id'].nunique()

    vals = [au, ret_users, new_users, res_users, churned_users]
    return vals
  
  
  
  
### This takes one to many rows of growth accounting figures for a specific
### date and calculates the user quick ratio
### It is used when calculating standard or rolling quick ratio
def calc_user_qr(row, 
                 new_col = 'new_users', 
                 res_col = 'resurrected_users', 
                 churned_col = 'churned_users'
                ):
  
    # These three lines make sure the column is found in the row. If not it sets it to 0 
    new_users = row[new_col] if hasattr(row, new_col) and pd.notnull(row[new_col]) else 0
    res_users = row[res_col] if hasattr(row, res_col) and pd.notnull(row[res_col]) else 0
    churned_users = row[churned_col] if hasattr(row, churned_col) and pd.notnull(row[churned_col]) else 0
    
    # If churned users is < 0 then we calculate the quick ratio using the equation.
    # Otherwise, we can't divide by zero, so it's set to NaN
    if churned_users < 0:
        user_qr = -1 * (new_users + res_users) / churned_users
    else:
        user_qr = math.nan
        
    return user_qr
  
  
  
  
### Using the numbers in the "final" growth accounting dataframe, calculate
### the number of users at the beginning of the period (BOP), the  
### period-over-period user retention ratio, the user quick ratio, and
### compound growth rates
def calc_user_ga_ratios(user_xga_df, time_period, growth_rate_periods = 12):
    
    time_fields = get_time_period_dict(time_period)
    frequency = time_fields['frequency']
    per = time_fields['period_abbr']

    this_ratio_df = user_xga_df.copy()

    # The users at the beginning of the period (BOP) are the active users from
    # the previous period. The shift function in Pandas allows us to get the
    # value from the previous row
    this_ratio_df['Users BOP'] = this_ratio_df[frequency + ' Active Users'].shift(1)
    
    # Period over Period User Retention is this period's Retained Users divided
    # by the number of users at the beginning of the period (BOP, above)
    this_ratio_df[per + 'o' + per + ' User Retention'] = this_ratio_df['Retained Users'] / this_ratio_df['Users BOP']
    
    # Call the calc_user_qr function to calculate each period's quick ratio,
    # passing in the correct column names
    this_ratio_df['User Quick Ratio'] = this_ratio_df.apply(lambda x: calc_user_qr(x, 
                 new_col = 'New Users', res_col = 'Resurrected Users', 
                 churned_col = 'Churned Users'), axis = 1)

    # Set the column name for the compound growth rate. For example, CMGR12 is
    # the compound monthly growth rate calculated over a 12 month period.
    cgr_col = 'User C%sGR%s' % (per, growth_rate_periods)
    
    # To calculate the compound growth rate, divide the active users from 
    # the current period by the active users from growth_rate_periods ago,
    # take it to the 1/growth_rate_periods power, and subtract one.
    this_ratio_df[cgr_col] = (np.power((this_ratio_df[frequency + ' Active Users'] / 
                                        this_ratio_df[frequency + ' Active Users']
                                        .shift(growth_rate_periods)), 
                                       1/growth_rate_periods)
                              - 1)
    
    # The Growth Threshold and growth rate target are constants for display 
    # purposes
    this_ratio_df['Growth Threshold'] = 1.0
    this_ratio_df[cgr_col + ' Target'] = 0.1

    return this_ratio_df
  
  


### Produces the "final" growth accounting dataframe with both user and
### revenue numbers for each time period in the "decorated" dataframe
def create_user_growth_accounting_df(xau_decorated_df, 
                                     time_period, 
                                     keep_last_period = True, 
                                     date_limit = None,
                                     growth_rate_periods = 12
                                    ):
  
    print('Creating User Growth Accounting dataframe')

    # These are the parameters that are set from the get_time_period_dict 
    # function above
    time_fields = get_time_period_dict(time_period)
    grouping_col = time_fields['grouping_col']
    first_period_col = time_fields['first_period_col']
    frequency = time_fields['frequency']
    
    # In this section, we create a copy of xau_decorated to join to itself.
    # We turn the Next_<period> column into the column to join with the current
    # period column. This results in user_id level data from this period,
    # the fields with a .t suffix, and last period, with a .l suffix.
    xau_decorated_df_last = xau_decorated_df.copy()
    xau_decorated_df_last[grouping_col + '_join'] = xau_decorated_df_last['Next_' + grouping_col]
    xau_decorated_df[grouping_col + '_join'] = xau_decorated_df[grouping_col]
    
    interim_join_cols = ['user_id', grouping_col + '_join']
    
    xga_interim = pd.merge(xau_decorated_df, xau_decorated_df_last, 
                           suffixes = ['.t', '.l'],
                           how = 'outer', 
                           left_on = interim_join_cols, 
                           right_on = interim_join_cols)
    
    # In this section we no longer group by user_id, and just group by the
    # time period (week or month). For each period grouping, we use calc_user_ga
    # to determine the numbers of active, retained, new, resurrected, and churned
    # users.
    groupby_cols = [grouping_col + '_join']
        
    user_xga = (xga_interim.groupby(groupby_cols)
                .apply(lambda x: pd.Series(calc_user_ga(x, 
                                                        grouping_col, 
                                                        first_period_col),
                                           index = [frequency + ' Active Users', 
                                                    'Retained Users', 
                                                    'New Users', 
                                                    'Resurrected Users',
                                                    'Churned Users'
                                                    ]))
                .reset_index()
                .rename(columns = {grouping_col + '_join' : grouping_col}))
            
    # Filter out any months that have 0 active users  
    user_xga = user_xga[user_xga[frequency + ' Active Users'] > 0]

    # If we choose not to keep the last period of the resultant data set, we can 
    # get rid of it. This is particularly helpful if you are running this to 
    # determine monthly growth accounting with data through a partial last month
    if not keep_last_period:
        user_xga = user_xga[:-1]
        
    # If we choose to eliminate more than one month of data from the end of the
    # dataset we can use the date_limit parameter. This comes into play if a 
    # dataset contains data with future dates, perhaps from projections
    if date_limit is not None:
        user_xga = user_xga[user_xga[grouping_col] <= date_limit]
        
    # This line calls the calc_user_ga_ratios function above to calculate
    # the ratios from the basic user counts in each category
    user_xga_with_ratios = calc_user_ga_ratios(user_xga, time_period, 
                                               growth_rate_periods)

    return user_xga_with_ratios


  
  
# Call create_user_growth_accounting_df, passing in mau_decorated and 'month'
# as the time period. We will use the default values for the other parameters.
user_xga_with_ratios = create_user_growth_accounting_df(mau_decorated, 'month')
user_xga_with_ratios.tail(10)


Creating User Growth Accounting dataframe


,Month_Year,Monthly Active Users,Retained Users,New Users,Resurrected Users,Churned Users,Users BOP,MoM User Retention,User Quick Ratio,User CMGR12,Growth Threshold,User CMGR12 Target
36,2018-05,3253,2507,441,305,-571,3078.0,0.814490,1.306480,0.032119,1.0,0.1
37,2018-06,3183,2626,340,217,-627,3253.0,0.807255,0.888357,0.025316,1.0,0.1
38,2018-07,3377,2691,386,300,-492,3183.0,0.845429,1.394309,0.033163,1.0,0.1
39,2018-08,3570,2835,453,282,-542,3377.0,0.839503,1.356089,0.036383,1.0,0.1
40,2018-09,3561,2947,383,231,-623,3570.0,0.825490,0.985554,0.036500,1.0,0.1
41,2018-10,3972,3034,581,357,-527,3561.0,0.852008,1.779886,0.038371,1.0,0.1
42,2018-11,4102,3307,518,277,-665,3972.0,0.832578,1.195489,0.036690,1.0,0.1
43,2018-12,4522,3482,740,300,-620,4102.0,0.848854,1.677419,0.042951,1.0,0.1
44,2019-01,4689,3811,583,295,-711,4522.0,0.842769,1.234880,0.040467,1.0,0.1
45,2019-02,4660,3865,540,255,-824,4689.0,0.824270,0.964806,0.038426,1.0,0.1


## 3. Load the transformed data into Google Sheets 
**Note**: For a more detailed discussion about loading data into Google Sheets, complete with inline code, visit [Section 3 of Mini-Pipeline: Cohort Analysis](https://colab.research.google.com/drive/1oYy-wJl6VZFgOsv8uw7iGChQxUjrR5rf#scrollTo=hzG-fUwkyM9K)
### Establish connection to Google Sheets for writing output files
The first time you run the cell below, or after some time of inactivity, you will be asked to click on a link. That link will take you to a new tab that will authorize this script to write to Google Sheets spreadsheets in your Google Account. To enable this feature, copy the code you get into the box below and hit Enter.

In [0]:
gc = tvcl.google_authenticate()

In [0]:
### Be sure to set this value to refer to your Google Sheets workbook
GOOGLE_SHEET_KEY = '1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0' 

In [0]:
print('https://docs.google.com/spreadsheets/d/' + GOOGLE_SHEET_KEY)

https://docs.google.com/spreadsheets/d/1-XnO_eWkRwX-E1fiA2Jkbe3kJvoyoPFsdeW7vnF6zS0


In [0]:
### Execute this function to write the data in the dataframe to the google sheet
### and tab name specified using the gc Google credentials
tvcl.write_to_google_sheet(user_xga_with_ratios, 
                           GOOGLE_SHEET_KEY, 
                           'MAU Growth Accounting', 
                           gc)

## 4. Visualize insights in Google Data Studio
A Google DataStudio dashboard preconfigured to read from the Google Sheet created above to visualize the data is [available at this link](https://datastudio.google.com/open/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV) or by clicking on the Google DataStudio logo at the bottom of the chart embedded below. It is available in read-only mode for you to copy, link to your own Google Sheet tabs, and see your own data visualized.
### 4.1 Show MAU
The first chart shows MAU with the three main active categories broken out:

- Retained Users are in blue
- New Users are in light green
- Resurrected Users are in darker green

Together these three categories add up to the total MAU for each month. The line snaking through the chart is the compound monthly growth rate calculated over a 12-month period (CMGR12). We see that it starts off high when the growth starts from a low base. But over time, the CMGR12 stabilizes around 4%. The User GMGR12 Target line of 10% serves as an example of a healthy growth rate for a successful startup, though this can vary depending on the situation.

In [0]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/YVKk', 
       width=600, 
       height=450)

### 4.2 Show MAU Growth Accounting
The chart below is similar to the last one in that it still shows the same New and Resurrected user counts in light green and darker green, respectively. This chart removes the retained user count in favor of churned users in orange (expressed as a negative number). It also removes the CMGR12 line in favor of two new lines: the User Quick Ratio (in black) and MoM User Retention rate (in blue). 

- User Quick Ratio = -1 * (New Users + Resurrected Users) / Churned Users
  - In other words, it is the ratio of the green bars at the top and the orange bar on the bottom
  - The horizontal line where the Quick Ratio = 1 serves as the demarcation line between growth and shrinkage
  - A Quick Ratio > 1 means that MAUs grew from last month to this month
  - A Quick Ratio < 1 means that MAUs shrank from last month to this month
  - For more about the Quick Ratio, visit our blog post [Quick Ratio as a Shortcut to Understand Product Growth](https://medium.com/theventurecity/quick-ratio-as-a-shortcut-to-understand-product-growth-ae60212bd371)
- MoM Retention Rate = Retained Users / Users BOP
  - Users BOP ("beginning of period") are the same as last month's users as calculated in the calc_user_ga_ratios function



In [0]:
IFrame('https://datastudio.google.com/embed/reporting/1xjS__Q6ZUXuUUARkgRvY4spYUw1ePksV/page/7eKk', 
       width=600, 
       height=450)